In [3]:
#essential libraries

#from google_drive import gdrive

from matplotlib.pyplot import show
import numpy as np
import pandas as pd
from scipy.cluster.vq import *
import operator
import matplotlib
reload(matplotlib)
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
import pickle
import shelve
import re
from collections import Counter, defaultdict, OrderedDict, deque
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
import scipy
from sklearn.feature_extraction import DictVectorizer
from matplotlib.backends.backend_pdf import PdfPages
import csv
import sys
import math
from copy import deepcopy, copy
import random
from datetime import datetime, timedelta
from operator import itemgetter
from itertools import chain
import os
import gc
import linecache
import multiprocessing
from functools import partial
from itertools import repeat
from IPython.display import clear_output
import json

import pprint
pp = pprint.PrettyPrinter()

import rdflib
from rdflib.namespace import OWL, RDF, RDFS
from rdflib import URIRef

from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import OneClassSVM, SVC
from sklearn.mixture import GMM
from sklearn.mixture import DPGMM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram
import scipy.cluster.hierarchy as hier
from scipy.cluster.hierarchy import fcluster
from scipy import spatial
from sklearn.multiclass import OneVsRestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
from scipy.cluster.hierarchy import linkage, dendrogram
import scipy.cluster.hierarchy as hier
from scipy import stats
from numpy.testing import assert_almost_equal

from scipy.spatial.distance import cosine as cosine_similarity

from divergence import gau_js as js_divergence
import brick_parser
reload(brick_parser)
from brick_parser import tagList, tagsetList, equipTagsetList, pointTagsetList, locationTagsetList,\
equalDict, pointTagList, equipTagList, locationTagList, equipPointDict
subTagListDict = dict([('point', pointTagList),
                          ('equip', equipTagList),
                          ('location', locationTagList)
                         ])
subTagsetListDict = dict([('point', pointTagsetList),
                          ('equip', equipTagsetList),
                          ('location', locationTagsetList)
                         ])

#from cmu_parser import cmu_building_parse

debugFlag = False

/home/jbkoh/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
INFO:rdflib:RDFLib Version: 4.2.1


Init Graph
Load Brick.ttl
Load BrickFrame.ttl
Init Graph
Load Brick.ttl
Load BrickFrame.ttl


In [4]:
import building_tokenizer
reload(building_tokenizer)
import building_tokenizer as toker

buildingName = 'ebu3b'
notUcsdBuildings = ['ghc']

tokenType = 'NoNumber'
if not buildingName in notUcsdBuildings:
    naeDict = dict()
    naeDict['bonner'] = ["607", "608", "609", "557", "610"]
    naeDict['ap_m'] = ['514', '513','604']
    naeDict['bsb'] = ['519', '568', '567', '566', '564', '565']
    naeDict['ebu3b'] = ["505", "506"]
    #naeDict['otterson'] = ["518", "517", "589", "590"]
    naeList = naeDict[buildingName]

    labeledFile = 'metadata/' + buildingName + '_sensor_types_location.csv'
    with open(labeledFile, 'rb') as fp:
        #truthDF = pd.read_excel(fp)
        truthDF = pd.DataFrame.from_csv(fp)
        #truthDF = truthDF.set_index(keys='Unique Identifier')

    wordFeatFile = 'data/wordfeat_'+buildingName+'.pkl'

    tokenTypeList = ['NoNumber', 'Alphanumeric', 'AlphaAndNum', 'NumAsSingleWord']

    bacnetTypeMapDF = pd.DataFrame.from_csv('metadata/bacnettype_mapping.csv')
    unitMap = pd.read_csv('metadata/unit_mapping.csv').set_index('unit')
    for val in Counter(unitMap.keys()).values():
        if val>1:
            "Unit map file ERROR"
            assert(False)
            
            
    trueDF = pd.DataFrame.from_csv('metadata/'+buildingName+'_sensor_types_location.csv')
    sensorDF, nameList, jcinameList, descList, unitList, bacnettypeList, wordList = \
    toker.structure_metadata(buildingName=buildingName, tokenType=tokenType, \
                             validSrcidList=trueDF.index.tolist(), withDotFlag=False)

    origSensorDF = deepcopy(sensorDF)
    origNameList = deepcopy(nameList)
    origJcinameList = deepcopy(jcinameList)
    origDescList = deepcopy(descList)
    origUnitList = deepcopy(unitList)
    origBacnettypeList = deepcopy(bacnettypeList)
    origWordList = deepcopy(wordList)

    #_, rawNameList, rawJcinameList, rawDescList, _, _, _ = toker.structure_metadata(buildingName=buildingName, tokenType='AlphaAndNum', \
    _, rawNameList, rawJcinameList, rawDescList, _, _, _ =\
    toker.structure_metadata(buildingName=buildingName, tokenType='JustSeparate', \
                             validSrcidList=trueDF.index.tolist(), withDotFlag=False)
    
    
else: 
    filename = 'metadata/'+buildingName+'_sensor_types_location.csv'
    #   filename = 'metadata/%s_sensor_types_location.csv'%buildingName
    df = pd.read_csv(filename)
    sentenceDict = dict()    
    for i,raw in enumerate(df['bas_raw'].tolist()):
        sentenceDict[i] = toker.tokenize(tokenType, raw)
    
    
## Common part
    
#with open('data/'+buildingName+'_str_score_dict.pkl', 'rb') as fp:
#    scoreDictDict = pickle.load(fp)
#    ### If a word is exactly matched with another, fix it.
#    for word, scoreDict in scoreDictDict.items():
#        if word in scoreDict.keys():
#            scoreDictDict[word] = {word:1}
#    scoreDictDict['co'] = {'co2':1}
        
adder = lambda x,y:x+y
splitter = lambda x:x.split()
bacnetTypes = list(set(reduce(adder,map(splitter,origBacnettypeList),[])))
#for bacnetType in bacnetTypes:
#    scoreDictDict[bacnetType] = {bacnetType:1}
#units = list(set(unitMap['word'].tolist()))
#for unit in units:
#    if type(unit)==str:
#        scoreDictDict[unit] = {unit:1}

In [6]:
with open("metadata/%s_sentence_dict.json"%buildingName, "r") as fp:
    sentenceDict = json.load(fp)

In [77]:
known_label_dict = {
    'nae':'none',
    'n': 'none',
    'ebu': 'building',
    'b': 'identifier', # b could be basement, which is ignored currently.
    'rm':'room',
    'vma':'vav',
    'newline':'none',
    'vnd':'vnd',
    'cmd':'command',
    'zn':'zone',
    'chilled_water_pump': 'chwp',
    'clgpid':'cooling_command',
    'htgpid':'heating_command',
    'rh':'reheat',
    'vlv':'valve',
    'supflo':'supply_air_flow',
    'sup':'supply',
    'rf':'return_fan',
    'clgmaxflo':'cooling_max_supply_air_flow_setpoint',
    'dx':'network_adapter',
    'hwp':'hot_water_pump',
    'sys':'system',
    'dpr':'damper',
    'chwp':'chilled_water_pump',
    'stpt':'setpoint',
    'clgminflo':'cooling_min_supply_air_flow_setpoint',
    'clg':'cooling',
    'dx':'network_adapter',
    'dmpr':'damper',
    'pos':'position',
    'actclgsp':'effective_cooling_temperature_setpoint',
    'clgminflo':'cooling_min_supply_air_flow_setpoint',
    'dpr':'damper',
    'clgmaxflo':'cooling_max_supply_air_flow_setpoint',
    'adj':'adjust',
    'commonsp':'temperature_setpoint',
    'htgflo':'heating_supply_air_flow_setpoint',
    'htg':'heating',
    'acthtgsp':'effective_heating_temperature_setpoint',
    'sen':'sensor',
    'supflosp':'supply_air_flow_setpoint',
    'dmprpos':'damper_position',
    'htgflow':'heating_supply_air_flow_setpoint',
    'sf':'supply_fan',
    'chilled_water',
    'alm':'alarm',
    
}
known_word_list = known_label_dict.keys()

In [78]:
def ask_expert_labels(sentence):
    # Input: List of words
    # Output: feature list:
    # Human operation: Give labels on the words.
    sliceSize = 2
    featureDict = dict()
    
    nonBrickTagList = ['none', 'identifier', 'network_adapter', 'vnd', 'unknown']

    #for words in rolling_window(sentence, 2):
    #    for word in wo
    print("\nGiven sentence: %s"%sentence)
    labelList = list()
    for word in sentence:
        if re.match('[^0-9a-zA-Z]+', word):
            labelList.append('none')
            continue
        if re.match('\d+', word):
            labelList.append('identifier')
            continue
        if word in known_word_list:
            labelList.append(known_label_dict[word])
            continue
        if word in tagList:
            labelList.append(word)
            continue
            
            
        while True:
            label = raw_input(str(word)+': \n')
            if label in tagList or label in tagsetList or label in nonBrickTagList:
                labelList.append(label)
                break
            else:
                print("Pick Tag among the following tags: " + str(tagList))
    return labelList

In [8]:
splitter = lambda x:x.split(' ')
train_sents = list()
for row in zip(rawNameList, rawJcinameList, rawDescList):
    para = list()
    for ele in row:
        para.extend([x for x in ele.split(' ') if x!=''])
        para.append('\n')
    train_sents.append(para)

In [38]:
# Review labelListDict
reviewedLabelListDict = dict()

for srcid, labelLi
st in labelListDict.items():
    reviewedLabelList = list()
    for word, label in zip(sentenceDict[srcid], labelList):
        x = raw_input(word + "\t" + label)
        if x=='':
            reviewedLabelList.append(label)
        else:
            reviewedLabelList.append(x)
        
    

SyntaxError: invalid syntax (<ipython-input-38-f1b453c5c056>, line 4)

In [79]:
labelFilename = 'metadata/%s_label_dict.json'%buildingName
with open(labelFilename, 'r') as fp:
    labelListDict = json.load(fp)

In [80]:
labelSampleNum = 20
for i in random.sample(range(0,len(sensorDF)), labelSampleNum):
    srcid = sensorDF.iloc[i].name
    if not srcid in labelListDict.keys():
        labelListDict[srcid] = ask_expert_labels(sentenceDict[srcid])
        


Given sentence: [u'nae', u'05', u'n', u'2', u'1', u'vma', u'106', u'dmpr', u'pos', u'newline', u'ebu', u'3', u'b', u'.', u'rm', u'-', u'b', u'250', u'a', u'.', u'.', u'dmpr', u'-', u'pos', u'newline', u'damper', u'position', u'newline']
a: 
identifier

Given sentence: [u'nae', u'06', u'n', u'2', u'1', u'vma', u'105', u'occ', u'cmd', u'newline', u'ebu', u'3', u'b', u'.', u'rm', u'-', u'2240', u'.', u'.', u'occ', u'-', u'cmd', u'newline', u'occupied', u'command', u'newline']
occ: 
occupied
occ: 
occupied

Given sentence: [u'nae', u'05', u'n', u'2', u'1', u'vma', u'103', u'htgflow', u'newline', u'ebu', u'3', u'b', u'.', u'rm', u'-', u'b', u'225', u'.', u'.', u'htgflow', u'newline', u'occupied', u'htg', u'flow', u'newline']

Given sentence: [u'nae', u'06', u'n', u'2', u'1', u'vma', u'158', u'htgflow', u'newline', u'ebu', u'3', u'b', u'.', u'rm', u'-', u'3200', u'a', u'.', u'.', u'htgflow', u'newline', u'occupied', u'htg', u'flow', u'newline']
a: 
identifier

Given sentence: [u'nae', u'05'

In [81]:
print len(labelListDict)
with open(labelFilename, 'w') as fp:
    json.dump(labelListDict,fp)

182


In [ ]:
#
#with open(labelFilename, 'w') as fp:
#    json.dump(labelListDict,fp)